In [1]:
import MySQLdb
import pandas as pd
import db2048
import traceback
import json

In [2]:
db = MySQLdb.connect("localhost", "u2048", "u2048passord", init_command="use testDB")

In [3]:
#!sudo pip3 install flask



In [4]:
def transform_rating(df):
    n = len(df)
    data = df.to_dict()
    rating = []
    for i in range(n):
        rating.append({
            "place": i + 1,
            "name": data["NAME"][i],
            "score": data["SCORE"][i]
        })
    return rating

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/rating')
def rating():
    limit = int(flask.request.args.get('limit', 5))
    if bool(flask.request.args.get('json', "")):
        return json.dumps(transform_rating(db2048.get_rating(db, limit=limit)))
    else:
        return db2048.get_rating(db, limit=limit).to_html(index=False)

@app.route('/add_user')
def add_user():
    try:
        user_name = str(flask.request.args['name'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        user_id, new = db2048.add_user(db, name=user_name)
        return json.dumps({"success": True, "user_id": user_id, "new": new}), 200
    except Exception:
        return traceback.format_exc(), 500
    
@app.route('/get_current_version')
def get_current_version():
    try:
        user_id = int(flask.request.args['user_id'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        version = db2048.get_current_version(db, user_id=user_id)
        return json.dumps({"success": True, "version": version}), 200
    except Exception:
        return traceback.format_exc(), 500
    
@app.route('/add_play')
def add_play():
    try:
        user_id = int(flask.request.args['user_id'])
        score = float(flask.request.args['score'])
        version = int(flask.request.args['version'])
    except Exception:
        return traceback.format_exc(), 400
    try:
        play_id, success = db2048.add_play(db, user_id=user_id, score=score, version=version)
        return json.dumps({"success": success, "play_id": play_id}), 200
    except Exception:
        return traceback.format_exc(), 500


@app.route('/')
def index():
    return("Hello World!")


app.run(port=8890, host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8890/ (Press CTRL+C to quit)
109.252.84.243 - - [14/Jan/2019 20:01:13] "GET /rating?json=true&limit=100 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:01:23] "GET /rating?json=true&limit=100 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:04:58] "GET /rating?json=true&limit=100 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:02] "GET /add_user?name=Yuri HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:02] "GET /get_current_version?user_id=12 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:02] "GET /add_play?user_id=12&version=1&score=354 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:02] "GET /rating?json=true&limit=100 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:18] "GET /add_user?name=Yuri HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:18] "GET /get_current_version?user_id=12 HTTP/1.1" 200 -
109.252.84.243 - - [14/Jan/2019 20:05:18] "GET /add_play?user_id=12&version=2&score=354 HTTP/1.1" 200 -
109.252.84.243

In [ ]:
import flask


In [10]:
help(app.run)


Help on method run in module flask.app:

run(host=None, port=None, debug=None, load_dotenv=True, **options) method of flask.app.Flask instance
    Runs the application on a local development server.
    
    Do not use ``run()`` in a production setting. It is not intended to
    meet security and performance requirements for a production server.
    Instead, see :ref:`deployment` for WSGI server recommendations.
    
    If the :attr:`debug` flag is set the server will automatically reload
    for code changes and show a debugger in case an exception happened.
    
    If you want to run the application in debug mode, but disable the
    code execution on the interactive debugger, you can pass
    ``use_evalex=False`` as parameter.  This will keep the debugger's
    traceback screen active, but disable code execution.
    
    It is not recommended to use this function for development with
    automatic reloading as this is badly supported.  Instead you should
    be using the :command

[{'place': 1, 'name': 'Happy Year', 'score': 81.0},
 {'place': 2, 'name': 'Testing Vasya', 'score': 64.0},
 {'place': 3, 'name': 'Testing Vasya', 'score': 32.0},
 {'place': 4, 'name': 'Happy Year', 'score': 27.0},
 {'place': 5, 'name': 'Testing Vasya', 'score': 16.0}]

In [4]:
len(db2048.get_rating(db, limit=5))

5

In [ ]:
!jupyter nbconvert server.ipynb --help


In [ ]:
!ls